In [ ]:
import numpy as np
import cv2
import math

# Your existing data
inv_poly_param = [
    889.91582358,
    528.72413142,
    -20.52846684,
    67.73730056,
    48.02888563,
    -14.14051173,
    20.22746961,
    38.15325087,
    -6.98641569,
    -26.68100659,
    -12.41222046,
    -1.83261524,
]
poly_param = [-5.72041404e02, 0.0, 6.76008376e-04, -3.97814203e-07, 5.05496647e-10]
affine_param = [
    9.99949231e-01,
    -4.44517298e-04,
    6.30993143e-04,
    9.68732891e02,
    7.48514758e02,
]

# Read your image (1920x1536 expected)
img = cv2.imread("./assets/SurCam01.jpeg", cv2.IMREAD_COLOR)
assert img is not None, "Image not found. Check the path."

# Pinhole model: 1280×960, ~90° horizontal FOV
Wp, Hp = 1920, 960
FOV = 120
fx = (Wp / 2.0) / math.tan(math.radians(FOV / 2.0))
pinhole = dict(fx=fx, fy=fx, cx=Wp / 2.0, cy=Hp / 2.0, width=Wp, height=Hp)
fisheye = dict(fx=889.871, fy=889.916, cx=9.68732891e02, cy=7.48514758e02)
print(pinhole)
# Reproject & save
from fisheye_transform import fisheye_to_pinhole

pinhole_img = fisheye_to_pinhole(img, 1.1, pinhole, fisheye)
cv2.imwrite("pinhole_from_fisheye_assumek=1.png", pinhole_img)

In [2]:
import math
import torch
import torch.nn.functional as F


def fisheye_to_pinhole_torch(
    img_bgr_uint8,  # torch.uint8 [Hf, Wf, 3] 或 [Hf, Wf]
    inv_poly_param,  # list/np.array, len = N (你给的是 12)
    affine_param,  # [c, d, e, xc, yc]
    Wp=1920,
    Hp=960,
    FOV_deg=90.0,  # 目标针孔图尺寸与FOV
):
    """
    将单张鱼眼图（OCamCalib模型）重映射为针孔相机视图（等距FOV裁切）。
    返回 torch.uint8 的针孔图，形状 [Hp, Wp, 3] 或 [Hp, Wp]。
    """
    # ---- 1) 准备输入图像张量 ----
    if img_bgr_uint8.dim() == 2:
        Hf, Wf = img_bgr_uint8.shape
        C = 1
        img = img_bgr_uint8.unsqueeze(0).unsqueeze(0).float()  # [1,1,Hf,Wf]
    else:
        Hf, Wf, _ = img_bgr_uint8.shape
        C = 3
        img = img_bgr_uint8.permute(2, 0, 1).unsqueeze(0).float()  # [1,3,Hf,Wf]

    device = img.device

    # ---- 2) 针孔相机内参（按你的设定）----
    fx = (Wp / 2.0) / math.tan(math.radians(FOV_deg / 2.0))
    fy = fx
    cx = Wp / 2.0
    cy = Hp / 2.0

    # ---- 3) 为目标针孔图生成像素网格 (u,v) ----
    u = torch.linspace(0, Wp - 1, Wp, device=device)
    v = torch.linspace(0, Hp - 1, Hp, device=device)
    uu, vv = torch.meshgrid(u, v, indexing="xy")  # [Hp, Wp]

    # 将像素坐标变成针孔相机坐标系下的方向 (归一化前)
    x = (uu - cx) / fx
    y = (vv - cy) / fy
    z = torch.ones_like(x)

    # 单位化/为 OCamCalib 计算 theta 做准备
    M_norm = torch.sqrt(x * x + y * y)  # sqrt(x^2 + y^2)
    eps = 1e-6
    M_safe = torch.clamp(M_norm, min=eps)
    theta = -torch.atan2(z, M_safe)  # 注意负号，和你给的代码保持一致

    # ---- 4) 计算 rho(theta) = sum a_i * theta^i ----
    inv_poly = torch.tensor(inv_poly_param, dtype=x.dtype, device=device)  # [N]
    # 以 Horner 法则更高效稳定：
    rho = torch.zeros_like(theta)
    for a in reversed(inv_poly):
        rho = rho * theta + a

    # ---- 5) 根据 OCamCalib 仿射把方向映射到鱼眼像素 (u_f, v_f) ----
    c, d, e, xc, yc = [float(t) for t in affine_param]
    # 先把 [x,y] 方向单位化，再乘以 rho
    x_dir = x / M_safe * rho
    y_dir = y / M_safe * rho

    # 2x2 仿射 + 主点
    u_f = c * x_dir + d * y_dir + xc
    v_f = e * x_dir + y_dir + yc

    # ---- 6) grid_sample 采样需要 [-1,1] 归一化坐标 ----
    # 使用 align_corners=True 时： norm = (coord / (size-1)) * 2 - 1
    grid_x = (u_f / (Wf - 1)) * 2.0 - 1.0
    grid_y = (v_f / (Hf - 1)) * 2.0 - 1.0
    grid = torch.stack([grid_x, grid_y], dim=-1).unsqueeze(0)  # [1,Hp,Wp,2]

    # ---- 7) 双线性采样，默认超界使用 0（黑色）----
    pinhole = F.grid_sample(
        img, grid, mode="bilinear", padding_mode="zeros", align_corners=True
    )  # [1,C,Hp,Wp]

    # ---- 8) 输出为 uint8 HxWxC ----
    pinhole = torch.clamp(pinhole, 0, 255).round().byte()
    if C == 1:
        out = pinhole.squeeze(0).squeeze(0)  # [Hp, Wp]
    else:
        out = pinhole.squeeze(0).permute(1, 2, 0)  # [Hp, Wp, 3]
    return out


# ---------------- 使用示例 ----------------
if __name__ == "__main__":
    import cv2
    import numpy as np

    inv_poly_param = [
        889.91582358,
        528.72413142,
        -20.52846684,
        67.73730056,
        48.02888563,
        -14.14051173,
        20.22746961,
        38.15325087,
        -6.98641569,
        -26.68100659,
        -12.41222046,
        -1.83261524,
    ]
    poly_param = [
        -5.72041404e02,
        0.0,
        6.76008376e-04,
        -3.97814203e-07,
        5.05496647e-10,
    ]  # 注：本流程不需要用到
    affine_param = [
        9.99949231e-01,
        -4.44517298e-04,
        6.30993143e-04,
        9.68732891e02,
        7.48514758e02,
    ]

    # 读取 1920x1536 的鱼眼图 (BGR)
    fisheye = cv2.imread("./assets/SurCam01.jpeg", cv2.IMREAD_COLOR)
    assert fisheye is not None and fisheye.shape[1] == 1920 and fisheye.shape[0] == 1536

    # 转 torch
    img_t = torch.from_numpy(fisheye)  # [H,W,3], uint8, BGR

    # 目标分辨率与 FOV
    Wp, Hp, FOV = 1920, 960, 140.0
    pinhole_bgr = (
        fisheye_to_pinhole_torch(
            img_t, inv_poly_param, affine_param, Wp=Wp, Hp=Hp, FOV_deg=FOV
        )
        .cpu()
        .numpy()
    )

    cv2.imwrite(f"rectified_pinhole_1920x960_FOV{FOV}.png", pinhole_bgr)
    print(f"saved -> rectified_pinhole_1920x960_FOV{FOV}.png")

saved -> rectified_pinhole_1920x960_FOV140.0.png


In [14]:
def resize_ocam_params(
    inv_poly_param,
    poly_param,
    affine_param,
    org_fisheye_size,
    target_fisheye_size,
    atol=1e-9,
):
    """
    Rescale OCamCalib-style parameters for a uniformly resized fisheye image.

    Parameters
    ----------
    inv_poly_param : sequence of float
        Coefficients of ρ(θ) used in your code (theta -> image radius in pixels).
    poly_param : sequence of float
        Coefficients of θ(ρ) (not used in your pipeline, but returned scaled for completeness).
    affine_param : sequence of float
        [c, d, e, xc, yc] as used by your undistortion code.
    org_fisheye_size : (int, int)
        (H_old, W_old) of the original fisheye image.
    target_fisheye_size : (int, int)
        (H_new, W_new) of the resized fisheye image.
    atol : float
        Tolerance to accept slight non-uniform differences.

    Returns
    -------
    inv_poly_new : list[float]
        Scaled ρ(θ) coefficients.
    poly_new : list[float]
        Scaled θ(ρ) coefficients.
    affine_new : list[float]
        Scaled [c, d, e, xc, yc].
    scale : float
        The uniform scale factor s that was applied.

    Notes
    -----
    - Requires uniform scaling: W_new/W_old ≈ H_new/H_old.
    - If you also crop, adjust (xc, yc) for the crop first, then apply this scaling.
    """
    H0, W0 = org_fisheye_size
    H1, W1 = target_fisheye_size
    sx = W1 / float(W0)
    sy = H1 / float(H0)

    if not np.isclose(sx, sy, atol=atol, rtol=0):
        raise ValueError(
            f"Non-uniform scaling not supported (sx={sx}, sy={sy}). "
            "Resize uniformly or reproject/recrop before scaling params."
        )

    s = sx  # uniform scale

    # 1) ρ(θ): multiply all coefficients by s
    inv_poly_new = [float(a) * s for a in inv_poly_param]

    # 2) θ(ρ): divide coeffs by s^i
    poly_new = [float(b) / (s**i) for i, b in enumerate(poly_param)]

    # 3) affine: keep c,d,e; scale principal point
    c, d, e, xc, yc = [float(v) for v in affine_param]
    affine_new = [c, d, e, xc * s, yc * s]

    return inv_poly_new, poly_new, affine_new, s


def resize_ocam_params_any(
    inv_poly_param, poly_param, affine_param, org_fisheye_size, target_fisheye_size
):
    """
    Rescale OCamCalib-style params for arbitrary (possibly non-uniform) resizing.

    Mapping used by your code:
        u = c*x' + d*y' + xc
        v = e*x' + 1*y' + yc
      where x' = (x/M) * rho(theta), y' = (y/M) * rho(theta).

    For a resize W_old x H_old  ->  W_new x H_new with
        sx = W_new/W_old,  sy = H_new/H_old,
    choose:
        rho'(theta) = sy * rho(theta)      (scale inv_poly_param by sy)
        c' = (sx/sy) * c,  d' = (sx/sy) * d,  e' = e
        xc' = sx * xc,  yc' = sy * yc
    Optionally, for theta(ρ) coefficients (if used):
        b_i' = b_i / sy**i

    Returns
    -------
    inv_poly_new : list[float]
    poly_new     : list[float]
    affine_new   : list[float]  (same layout [c, d, e, xc, yc])
    scales       : (sx, sy)
    """
    H0, W0 = org_fisheye_size
    H1, W1 = target_fisheye_size
    sx = float(W1) / float(W0)
    sy = float(H1) / float(H0)

    # ρ(θ): scale by sy
    inv_poly_new = [float(a) * sy for a in inv_poly_param]

    # θ(ρ): b_i' = b_i / sy^i  (not used by your pipeline but returned for completeness)
    poly_new = [float(b) / (sy**i) for i, b in enumerate(poly_param)]

    # affine: adjust as derived above
    c, d, e, xc, yc = [float(v) for v in affine_param]
    c_new = (sx / sy) * c
    d_new = (sx / sy) * d
    e_new = e
    xc_new = sx * xc
    yc_new = sy * yc
    affine_new = [c_new, d_new, e_new, xc_new, yc_new]

    return inv_poly_new, poly_new, affine_new, (sx, sy)


org_size = (1920, 1536)
new_size = (1920 // 2, 1536 // 2)

# inv_poly_new, poly_new, affine_new, s = resize_ocam_params(
#     inv_poly_param, poly_param, affine_param, org_size, new_size
# )

inv_poly_new, poly_new, affine_new, s = resize_ocam_params_any(
    inv_poly_param, poly_param, affine_param, org_size, new_size
)

print("Scaled inv_poly_param:", inv_poly_new)
print("Scaled poly_param:", poly_new)
print("Scaled affine_param:", affine_new)


# 读取 1920x1536 的鱼眼图 (BGR)
fisheye = cv2.imread("./assets/SurCam01.jpeg", cv2.IMREAD_COLOR)
print(fisheye.shape)
assert fisheye is not None and fisheye.shape[1] == 1920 and fisheye.shape[0] == 1536
fisheye = cv2.resize(fisheye, new_size, interpolation=cv2.INTER_LINEAR)
print(fisheye.shape)
# 转 torch
img_t = torch.from_numpy(fisheye)  # [H,W,3], uint8, BGR

# 目标分辨率与 FOV
Wp, Hp, FOV = 1920, 960, 140.0
pinhole_bgr = (
    # fisheye_to_pinhole_torch(img_t, inv_poly_param, affine_param, Wp=Wp, Hp=Hp, FOV_deg=FOV)
    fisheye_to_pinhole_torch(img_t, inv_poly_new, affine_new, Wp=Wp, Hp=Hp, FOV_deg=FOV)
    .cpu()
    .numpy()
)

cv2.imwrite(f"rectified_pinhole_1920x960_FOV{FOV}.png", pinhole_bgr)
print(f"saved -> rectified_pinhole_1920x960_FOV{FOV}.png")

Scaled inv_poly_param: [444.95791179, 264.36206571, -10.26423342, 33.86865028, 24.014442815, -7.070255865, 10.113734805, 19.076625435, -3.493207845, -13.340503295, -6.20611023, -0.91630762]
Scaled poly_param: [-572.041404, 0.0, 0.002704033504, -3.182513624e-06, 8.087946352e-09]
Scaled affine_param: [0.999949231, -0.000444517298, 0.000630993143, 484.3664455, 374.257379]
(1536, 1920, 3)
(768, 960, 3)
saved -> rectified_pinhole_1920x960_FOV140.0.png
